In [1]:
import sys

import nltk
nltk.download('stopwords')
from nltk.tokenize import RegexpTokenizer

[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


In [2]:
from nltk.corpus import stopwords
import numpy as np
from keras.utils import np_utils

Using TensorFlow backend.
C:\Users\Lenovo\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Lenovo\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Lenovo\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Lenovo\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of

In [3]:

from keras.models import Sequential
from keras.layers import Dropout,Dense,LSTM
from keras.callbacks import ModelCheckpoint

In [5]:
import requests
content =requests.get("http://www.gutenberg.org/cache/epub/11/pg11.txt").text
open("mydata.txt","w",encoding="utf-8").write(content)

167516

In [6]:
text=open("mydata.txt").read()

In [7]:
def tokenize_fi(file):
    file=file.lower()
    tokenizer=RegexpTokenizer(r'\w+')
    tokens =tokenizer.tokenize(file)
    filterated=filter(lambda token:token not in stopwords.words('english'),tokens)
    return"".join(filterated)

In [8]:
processed =tokenize_fi(text)

In [9]:
chars=sorted(list(set(processed)))
char_tonum=dict((c,i)for i,c in enumerate(chars))

In [10]:
input_len =len(processed)
out_len =len(chars)
print(input_len)
print(out_len)

77480
38


In [11]:
seq_len=100
x_data=[]
y_data=[]
for i in range(0,input_len-seq_len,1):
    in_se=processed[i:i+seq_len]
    out_se=processed[i+seq_len]
    x_data.append([char_tonum[chars] for chars in in_se])
    y_data.append([char_tonum[out_se]])
pattern=len(x_data)
print(pattern)

77380


In [12]:
x=np.reshape(x_data,(pattern,seq_len,1))
x=x/float(out_len)
y=np_utils.to_categorical(y_data)

In [13]:
model=Sequential()
model.add(LSTM(256,input_shape=(x.shape[1],x.shape[2]),return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256,return_sequences=True))
model.add(Dropout(0.2))               
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1],activation='softmax'))

In [14]:
model.compile(loss='categorical_crossentropy',optimizer='adam')

In [26]:
filepath="model_weigh_saved.hdf5"
model.load_weights(filepath)
check=ModelCheckpoint(filepath,monitor='loss',verbose=1,save_best_only=True)

In [16]:
desired=[check]
model.fit(x,y,batch_size=256,epochs=1,callbacks=desired)


Epoch 1/1
77380/77380 [==============================] - 1716s 22ms/step - loss: 2.9915

Epoch 00001: loss improved from inf to 2.99147, saving model to model_weigh_saved.hdf5


In [24]:
num_tochar=dict((i,c)for i,c in enumerate(chars))
start =np.random.randint(0,len(x_data)-1)
patern =x_data[start]
print("\"",''.join([num_tochar[values]for values in patern]),"\"")

" kytwinkletwinkledormouseshookbegansingingsleeptwinkletwinkletwinkletwinklewentlongpinchmakestopwellh "


In [20]:
for i in range(1000):
    x=np.reshape(patern,(1,len(patern),1))
    x=x/float(out_len)
    prediction=model.predict(x,verbose=0)
    index=np.argmax(prediction)
    result=num_tochar[index]
    seq_in=[num_tochar[value]for value in patern]
    sys.stdout.write(result)
    patern.append(index)
    patern=patern[1:len(patern)]

eeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee